In [1]:
import pandas as pd

In [ ]:
df12raw = pd.read_excel('RePORTER_PRJ_C_FY2012.xlsx')
df12 = df12raw
df15raw = pd.read_excel('RePORTER_PRJ_C_FY2012.xlsx')
df15 = df15raw

In [ ]:
# parse the IC names and costs into an array
def fundingSplitter(icArray):
    icArray = str(icArray).split('''\\''')
    extractedICs = []
    for ic in icArray:
        if ic:
            tmp = ic.split(''':''')
            icName = tmp[0] if tmp[0] else ''
            cost = tmp[1] if len(tmp)>1 else 0
            extractedICs.append({'IC': icName, 'Cost': cost})
    return extractedICs
df12['FUNDING_PARSED'] = df12.FUNDING_ICs.apply(fundingSplitter)
df15['FUNDING_PARSED'] = df15.FUNDING_ICs.apply(fundingSplitter)

<div style="background: PALETURQUOISE">
<div style="text-align: center">
<br>
<h1>ANALYSIS</h1>
<h4>Explore for relationships, correlations</h4>
<br>
</div></div>

### For measuring unique-ness of a column to all other columns in that DF

In [ ]:
df.drop(['FY','ARRA_FUNDED','FUNDING_PARSED','NIH_SPENDING_CATS_PARSED','PROJECT_TERMS_PARSED','PI_NAMES_PARSED','PI_IDS_PARSED'], axis=1)\
    .groupby('FULL_PROJECT_NUM')\
    .apply(lambda group: group.apply(lambda attr: attr.nunique()))\
    .apply(lambda aggdCol: float(aggdCol.sum()) / len(aggdCol))\
    .sort_values(ascending=False)

### For checking if a column has >1 unique value for each FULL_PROJECT_NUM

In [ ]:
df[['FULL_PROJECT_NUM','FUNDING_ICs']]\
    .groupby('FULL_PROJECT_NUM')\
    .apply(lambda proj: proj['FUNDING_ICs'].nunique())\
    .sort_values(ascending=False)

### Define series.map helper functions

In [4]:
# df.Series map function to help filter based on participating in funding a grant
def filterIC(arr, IC):
    for ic in arr:
        return True if ic['IC'] == IC else False

### Detemine Obligated Funds of NHLBI in FY12, FY15

In [6]:
# df.Series map function to determine an IC's participating in a grant funding
def costExtraction(arr, IC):
    for ic in arr:
        if ic['IC']==IC:
            return int(ic['Cost'])
        else:
            return 0
print 'FY12', df12.FUNDING_PARSED.map(lambda r: costExtraction(r, 'NHLBI')).sum()/1E9
print 'FY15', df15.FUNDING_PARSED.map(lambda r: costExtraction(r, 'NHLBI')).sum()/1E9

FY12

NameError: name 'df12' is not defined

### Determine Cross-Institute Grants Funding Collaboration

In [8]:
# collect unique list of IC Abbreviations
ICs = []
for ic in df12.FUNDING_PARSED.tolist():
    for i in ic:
        if i['IC']:
            ICs.append(i['IC'])
IClist = list(set(ICs))

# runs calculation to return % of grants funded that an IC led
def calc(ICNAME):
    return [float(df12[df12.FUNDING_PARSED.map(lambda r: filterIC(r, ICNAME))].IC_NAME.value_counts().tolist()[0])\
    /df12[df12.FUNDING_PARSED.map(lambda r: filterIC(r, ICNAME))].IC_NAME.value_counts().sum()*100, \
    df12[df12.FUNDING_PARSED.map(lambda r: filterIC(r, ICNAME))].IC_NAME.value_counts()]

# print results on syndicated grants
for i in IClist:
    x = calc(i)
    print i
    print x[0]
    print x[1]
    print '\n'

NameError: name 'df12' is not defined